<a href="https://colab.research.google.com/github/lekiho/NSR_AD/blob/main/LSTM_ED_WADI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'

Mounted at /content/drive


In [4]:
# !git clone https://github.com/KDD-OpenSource/DeepADoTS.git

/content/drive/MyDrive/Colab_Notebooks/DeepADoTS


In [ ]:
# !pip install -r requirements.txt --target='/content/drive/MyDrive/Colab_Notebooks/DeepADoTS/enviro'
# !pip install tensorflow==1.15.5 --target='/content/drive/MyDrive/Colab_Notebooks/DeepADoTS/enviro'
# !pip install donut --target='/content/drive/MyDrive/Colab_Notebooks/DeepADoTS/enviro'

In [14]:
%cd /content/drive/MyDrive/Colab_Notebooks/DeepADoTS
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/DeepADoTS')
import enviro
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import roc_auc_score

from src.algorithms import lstm_enc_dec_axl
# from src.datasets import Dataset

/content/drive/MyDrive/Colab_Notebooks/DeepADoTS


In [15]:
train = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/GTA/data/rename/WADI_14days_colab.csv', index_col = [0])
test = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/GTA/data/rename/WADI_attackdata_colab.csv', index_col = [0])

In [33]:
x_train = train.drop('Timestamp', axis=1)
x_test = pd.DataFrame(np.ones(len(x_train)))

In [37]:
y_train = test.drop(['Timestamp','label'],axis=1)
y_test = pd.DataFrame(test['label'])

In [42]:
# x_train, y_train, x_test, y_test = lstm_enc_dec_axl().data()

In [261]:
model = lstm_enc_dec_axl.LSTMED(num_epochs=20,batch_size=5000,lr=0.001, hidden_size=16)

In [262]:
train = x_train.set_index('1_AIT_001_PV')

In [263]:
# model.fit(x_train)
model.fit(train)

  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
100%|██████████| 20/20 [27:36<00:00, 82.84s/it]
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [189]:
test = y_train.set_index('1_AIT_001_PV')

In [265]:
score = model.predict(test)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [300]:
print(roc_auc_score(y_test, score))

0.4935744257805826


In [301]:
y_test.value_counts()

label
 1       162824
-1         9977
dtype: int64

In [829]:
def binarize(score, threshold=None):
        threshold = threshold if threshold is not None else threshold(score)
        score = np.where(np.isnan(score), np.nanmin(score) - sys.float_info.epsilon, score)
        return np.where(score >= threshold,1, -1)
def threshold(score):
        return np.nanmean(score) + 2 * np.nanstd(score)

In [845]:
thres = threshold(score)
# output = binarize(score, thres)
output = binarize(score, 500)

In [846]:
pd.DataFrame(output).value_counts()

 1    95890
-1    76911
dtype: int64

In [847]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
print(f1_score(y_test, output))
print(recall_score(y_test, output))
print(confusion_matrix(y_test, output))

0.7007429052931037
0.5567115412961234
[[ 4733  5244]
 [72178 90646]]


In [848]:
pd.DataFrame(output).to_csv('/content/drive/MyDrive/Colab_Notebooks/DeepADoTS/result/LSTM-ED_num_epochs=20,batch_size=5000,lr=0.001, hidden_size=16_threshold=500.csv')

In [722]:
from src.evaluation import Evaluator
def get_metrics_by_thresholds(y_test: list, score: list, thresholds: list):
        for threshold in thresholds:
            anomaly = binarize(score, threshold=threshold)
            metrics = Evaluator.get_accuracy_precision_recall_fscore(y_test, anomaly)
            yield (anomaly.sum(), *metrics)

def get_optimal_threshold(y_test, score, steps=1000, return_metrics=False):
        maximum = np.nanmax(score)
        minimum = np.nanmin(score)
        threshold = np.linspace(minimum, maximum, steps)
        metrics = list(get_metrics_by_thresholds(y_test, score, threshold))
        metrics = np.array(metrics).T
        anomalies, acc, prec, rec, f_score, f01_score = metrics
        if return_metrics:
            return anomalies, acc, prec, rec, f_score, f01_score, threshold
        else:
            return threshold[np.argmax(f_score)]

In [723]:
# anomalies, acc, prec, rec, f_score, f01_score, threshold = get_optimal_threshold(y_test,score,return_metrics=True)
thr = get_optimal_threshold(y_test, score)

In [724]:
thr

1283948928.481919

In [525]:
thres

1970676089.9390702

In [521]:
thres = threshold(score)
# output = binarize(score, thres)
output = binarize(score, 1000)

In [522]:
pd.DataFrame(output).value_counts()

 1    95096
-1    77705
dtype: int64

In [523]:
pd.DataFrame(y_test).value_counts()

label
 1       162824
-1         9977
dtype: int64

In [524]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
print(f1_score(y_test, output))
print(recall_score(y_test, output))
print(precision_score(y_test, output))
print(accuracy_score(y_test, output))
print(confusion_matrix(y_test, output))

0.7013259925558314
0.5554647963445193
0.9510704971817953
0.5542039687270328
[[ 5324  4653]
 [72381 90443]]


In [241]:
y_test.value_counts()

label
 1       162824
-1         9977
dtype: int64

In [298]:
pd.DataFrame(anomaly)

 1    172800
-1         1
dtype: int64

In [286]:
binar = binarize(score, thr)

In [287]:
pd.DataFrame(binar).value_counts()

 1    172800
-1         1
dtype: int64